# Mechanism Manipulation

MeshCatMechanisms.jl can use [InteractBase.jl](https://github.com/piever/InteractBase.jl) to let you interactively modify the configuration of a mechanism state or a mechanism's visualizer. We'll show off a few use cases here. 

In [ ]:
# Import our packages
using MeshCatMechanisms
using RigidBodyDynamics

In [ ]:
# Create a random mechanism and its associated visualizer

# seed the random number generator so we get repeatable results
import Random
Random.seed!(75)
mechanism = rand_chain_mechanism(Float64, 
    [QuaternionFloating{Float64}; [Revolute{Float64} for i = 1:5]]...)
mvis = MechanismVisualizer(mechanism, Skeleton(randomize_colors=true))

In [ ]:
# Render the visualizer
IJuliaCell(mvis)

The simplest thing we might want to do is to manipulate a visualized mechanism. To do that, we simply need the `manipulate!()` function: 

In [ ]:
# Create sliders to manipulate the visualizer's configuration
widget = manipulate!(mvis)

The `manipulate!()` function can also do more than just adjust a visualizer. The low-level signature of `manipulate!()` is:

```julia
manipulate!(callback::Function, state::RigidBodyDynamics.MechanismState)
```

This method does the following things: 
* Creates `InteractBase.slider`s for each joint in the mechanism
* Sets up observers (using Observables.jl) to watch for changes to those sliders
* On any slider change:
  * Update the configuration of the provided state
  * Call `callback(state)`
* Returns the collected sliders

The jupyter notebook itself helps by automatically displaying the resulting sliders when they are the final item in a notebook cell. To force the sliders to display, just use the `display()` function: 

```julia
display(manipulate!(...))
```

Let's try a simple example:

In [ ]:
state = MechanismState(mechanism)
manipulate!(state) do x
    @show configuration(x)
end

By the way, if you're not familiar with the way `do` blocks work in Julia, see <https://docs.julialang.org/en/stable/manual/functions/#Do-Block-Syntax-for-Function-Arguments-1>

With that in mind, we can easily reconstruct the behavior of `manipulate!()` when we pass it a `MechanismVisualizer` instead of a state and a callback: 

In [ ]:
# manipulate!(mvis) does essentially the following: 

manipulate!(state) do x
    set_configuration!(mvis, configuration(x))
end